In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, ElasticNetCV
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error
from sklearn.cluster import KMeans, DBSCAN, OPTICS
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

from statsmodels.regression.linear_model import OLS
#import matplotlib.pyplot as plt
from scipy.stats import norm
import random

# suppress all warnings
import warnings
warnings.filterwarnings("ignore")

# Import Data

In [4]:
df = pd.read_csv('/Users/arthur.pentecoste/ProjectRA/Demand_Paper/Code_DAC/33_cleaned_data.csv')
df['trend'] = df.week // 1000 - 2013
df['promo'] = df.promoFlag.astype('int')
df

,sku,class,week,desc,units,revenue,promoFlag,promoPrice,forecast,functionality,color,vendor,fatigue,month,price,trend,promo
0,792257,335,2013111,LOGITECH WIRELESS MK320,366,10889.71,False,NaN,NaN,keyboard,black,Logitech,0,11,29.753306,0,0
1,792257,335,2013112,LOGITECH WIRELESS MK320,424,12453.43,False,NaN,NaN,keyboard,black,Logitech,0,11,29.371297,0,0
2,792257,335,2013113,LOGITECH WIRELESS MK320,193,5635.89,False,NaN,NaN,keyboard,black,Logitech,0,11,29.201503,0,0
3,792257,335,2013114,LOGITECH WIRELESS MK320,128,3662.98,False,NaN,NaN,keyboard,black,Logitech,0,11,28.617031,0,0
4,792257,335,2013115,LOGITECH WIRELESS MK320,145,4323.56,False,NaN,NaN,keyboard,black,Logitech,0,11,29.817655,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19059,953406,335,2016085,SCULPT TOUCH MOUSE BLUETOOTH,1,16.99,False,NaN,NaN,mouse,blue,Microsoft,52,8,16.990000,3,0
19060,953406,335,2016091,SCULPT TOUCH MOUSE BLUETOOTH,1,16.99,False,NaN,NaN,mouse,blue,Microsoft,53,9,16.990000,3,0
19061,953406,335,2016092,SCULPT TOUCH MOUSE BLUETOOTH,1,16.99,False,NaN,NaN,mouse,blue,Microsoft,54,9,16.990000,3,0
19062,953406,335,2016101,SCULPT TOUCH MOUSE BLUETOOTH,1,16.99,False,NaN,NaN,mouse,blue,Microsoft,57,10,16.990000,3,0


In [5]:
df.month.unique()

array([11, 12,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [6]:
data = pd.get_dummies(data=df, columns=['functionality','color','vendor','month'])
data.columns

Index(['sku', 'class', 'week', 'desc', 'units', 'revenue', 'promoFlag',
       'promoPrice', 'forecast', 'fatigue', 'price', 'trend', 'promo',
       'functionality_CD sleeves', 'functionality_CD spindle',
       'functionality_cloud storage', 'functionality_dimmer',
       'functionality_external dvd', 'functionality_flash drive',
       'functionality_flash memory', 'functionality_hard drive',
       'functionality_headset', 'functionality_keyboard',
       'functionality_keyboard & mouse', 'functionality_mouse',
       'functionality_switch', 'functionality_tracker', 'functionality_webcam',
       'functionality_wristband', 'color_black', 'color_blue',
       'color_burgundy', 'color_green', 'color_grey', 'color_harlequin',
       'color_metal', 'color_multicolor', 'color_none', 'color_none ',
       'color_pink', 'color_purple', 'color_red', 'color_silver', 'color_teal',
       'color_violet', 'color_white', 'vendor_Belkin', 'vendor_Fitbit',
       'vendor_HP', 'vendor_Lexar', 'ven

In [7]:
skuSet = df.sku.unique()
skuData = {}
colnames = list(data.columns)
for i in skuSet:
    df_i = data[data.sku == i]
    skuData[i] = {'X': df_i[colnames[9:]].values,
                  'y': df_i.units.values}

In [8]:
skuSet.size

147

# Data preparation and linear regressions (OLS models)

## Decentralized

In [9]:
# testing data dictionary
X_dict = {}
y_dict = {}

skuModels = {}
y_pred = []
y_test = []
y_train = []


train_size = 0
test_size = 0
range_dict = {}
row_train = 0
row_test = 0

for i in skuSet:
    X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(
          skuData[i]['X'], skuData[i]['y'], test_size = 0.33, random_state = 5 )
    
    X_dict[i] = {'train': X_train_i, 'test': X_test_i}
    y_dict[i] = {'train': y_train_i, 'test': y_test_i}
    
    train_size += y_train_i.size
    test_size += y_test_i.size
    range_dict[i] = {'train': (row_train, row_train + y_train_i.size), 
                     'test':  (row_test, row_test + y_test_i.size) }
    
    row_train += y_train_i.size
    row_test += y_test_i.size
    
    model_i = OLS(y_train_i, X_train_i, hasconst = False)
    skuModels[i] = model_i.fit()
    y_pred += list(skuModels[i].predict(X_test_i))
    y_test += list(y_test_i)
    y_train += list(y_train_i)

y_train = np.array(y_train)
y_test = np.array(y_test)

print('R2:',r2_score(y_test, np.array(y_pred)))
print('MSE:', mean_squared_error(y_test, np.array(y_pred)))
print('Number of coeff:', X_train_i.shape[1]*147)



R2: 0.3548464172551853
MSE: 72992.09233816856
Number of coeff: 9408


## Centralized

In [10]:
X_cen_train = X_dict[skuSet[0]]['train'] 
X_cen_test = X_dict[skuSet[0]]['test']

for sku in skuSet[1:]:
    X_cen_train = np.concatenate((X_cen_train, X_dict[sku]['train']), axis = 0)
    X_cen_test = np.concatenate((X_cen_test, X_dict[sku]['test']), axis = 0)

model_cen = LinearRegression(fit_intercept=False).fit(X_cen_train, y_train)
print(r2_score(y_test, model_cen.predict(X_cen_test)))  
print('MSE:', mean_squared_error(y_test, model_cen.predict(X_cen_test)))

0.16262763426116278
MSE: 94739.55144354519


# DAC

## Fonction learn_structure

In [11]:
def learn_structure(theta = 0.01, 
                    upp = 0.9, 
                    low = 0.1,
                    num_clusters = 9,
                    print_structure = False):
    
    d = skuData[skuSet[0]]['X'].shape[1]
    n = skuSet.size
    aggre_level = []
    clus_columns = []
    n_cols_alg = 0
    z = num_clusters
    all_coeff = np.zeros((n,d))
    all_coeff[0,:] = skuModels[skuSet[0]].params


    for j in range(d):

        # a n-1 vector recording if two betas have the same mean
        test_j = np.zeros(n-1)

        for i in range(1,n):
            sku = skuSet[i]
            all_coeff[i,j] = skuModels[sku].params[j]

            z_stat = ( np.abs(skuModels[skuSet[0]].params[j] - skuModels[sku].params[j]) / 
                      np.sqrt(np.square(skuModels[skuSet[0]].bse[j]) + np.square(skuModels[sku].bse[j])) )
            p_value = 1 - norm.cdf(z_stat)
            if p_value >= theta:
                test_j[i-1] = 1

        if print_structure:
            print('Feature:', colnames[j+9])
            print('Ratio:', np.mean(test_j))

        if np.sum(test_j) >= upp*(n-1):
            aggre_level.append('dept')
            n_cols_alg += 1

        elif np.sum(test_j) <= low*(n-1):
            aggre_level.append('sku')
            n_cols_alg += n

        else:
            aggre_level.append('clus')
            clus_columns.append(j)
            n_cols_alg += z

        if print_structure:
            print(aggre_level[-1])
            print()

    if len(clus_columns) > 0:
        X_clus = all_coeff[:, clus_columns]
        kmeans = KMeans(n_clusters=z, random_state=0).fit(X_clus)

    X_alg_train = np.zeros((train_size, n_cols_alg))
    X_alg_test = np.zeros((test_size, n_cols_alg))

    count = 0
    for i in range(d):
        if aggre_level[i] == 'dept':

            for sku in skuSet:
                # find the corresponding range
                idx_train = range_dict[sku]['train']
                idx_test = range_dict[sku]['test']

                # stack the data
                X_alg_train[idx_train[0]:idx_train[1], count] = X_dict[sku]['train'][:,i]
                X_alg_test[idx_test[0]:idx_test[1], count] = X_dict[sku]['test'][:,i]

            count += 1

        elif aggre_level[i] == 'clus':

            for j in range(z):
                # the indices of items in cluster j
                clus_items = list(np.where(kmeans.labels_ == j)[0])
                for idx in clus_items:
                    sku = skuSet[idx]
                    # find the corresponding range
                    idx_train = range_dict[sku]['train']
                    idx_test = range_dict[sku]['test']

                    # stack the data
                    X_alg_train[idx_train[0]:idx_train[1], count] = X_dict[sku]['train'][:,i]
                    X_alg_test[idx_test[0]:idx_test[1], count] = X_dict[sku]['test'][:,i]


                count += 1
        else:
            for sku in skuSet:
                # find the corresponding range
                idx_train = range_dict[sku]['train']
                idx_test = range_dict[sku]['test']

                # stack the data
                X_alg_train[idx_train[0]:idx_train[1], count] = X_dict[sku]['train'][:,i]
                X_alg_test[idx_test[0]:idx_test[1], count] = X_dict[sku]['test'][:,i]

                count += 1
        
    
    return X_alg_train, X_alg_test



def mape(y_true, y_pred): 

    return np.median(np.abs((y_true - y_pred) / y_true))

## Base model

In [12]:
z = 4
upp, low = 0.8, 0.1
X_alg_train, X_alg_test = learn_structure(upp = 0.8, low = 0.1, 
                                          num_clusters = z, print_structure = False)
# When print structure use seed = 2
print('Fitting model...')

scaler = MinMaxScaler()
#X_alg_train = scaler.fit_transform(X_alg_train)
#X_alg_test = scaler.transform(X_alg_test)
model_0 = LinearRegression().fit(X_alg_train, y_train)
#model_1 = Lasso(fit_intercept = False, alpha = 0.0001).fit(X_alg_train, y_train)
print('Parameters:', upp, low, z) 
print('R2 score:', r2_score(y_test, model_0.predict(X_alg_test)))
print('MSE:', mean_squared_error(y_test, model_0.predict(X_alg_test)))
print('Number of coeff:', X_alg_train.shape[1])
print('MAPE:', mape(y_test, model_0.predict(X_alg_test)))

Fitting model...
Parameters: 0.8 0.1 4
R2 score: 0.35500518753160026
MSE: 72974.12921281888
Number of coeff: 5478
MAPE: 1.3419361943783967


In [13]:
print(X_alg_train.shape[0] + X_alg_test.shape[0])
print((np.sum(y_train) + np.sum(y_test)) / (y_train.size + y_test.size))
print(np.mean(data.units))
print(np.std(data.units))
print(np.mean(data.price))
print(np.std(data.price))
print(np.mean(data.promoFlag))
1 - (np.mean(data.price[data.promoFlag]) / np.mean(data.price[~data.promoFlag]))

19064
108.11356483424255
108.11356483424255
377.44829018995057
42.915910211040874
38.803359572724155
0.3126835921107847


0.042050977236539944

## Selection of hyperparameters

In [0]:
i=0
params=[]
maximum1=0

for upp in [0.7,0.8,0.9]:
    for low in [0.1,0.2,0.3]:
        for z in [4,5,6,7,8,9,10]:
          i+=1
          X_alg_train, X_alg_test = learn_structure(upp = upp, low = low, num_clusters = z)
          print('Fitting model',i,'/63')
          model_0 = LinearRegression(fit_intercept=False).fit(X_alg_train, y_train)
          print('  Parameters:', upp, low, z)
          score=r2_score(y_test, model_0.predict(X_alg_test))
          print('  R2 score:', score)
          print('  MAPE:', mape(y_test, model_0.predict(X_alg_test)))

          if score>maximum1:
            params = [upp,low,z]
            maximum1 = score

print('\nBest Model:')
print('parameters:',params)
print('R2:',maximum1)  

Fitting model 1 /63
  Parameters: 0.7 0.1 4
  R2 score: 0.33244084125139073
  MAPE: 1.6658421566611843
Fitting model 2 /63


In [14]:
i=0
params=[]
maximum1=0

for upp in [0.7,0.8,0.9]:
    for low in [0.1,0.2,0.3]:
        for z in [4,5,6,7,8,9,10]:
          i+=1
          X_alg_train, X_alg_test = learn_structure(upp = upp, low = low, num_clusters = z)
          print('Fitting model',i,'/63')
          model_0 = LinearRegression(fit_intercept=False).fit(X_alg_train, y_train)
          scores=cross_val_score(estimator=model_0, X=X_alg_train, y=y_train, cv=5, scoring='r2')
          score=np.average(scores)
          if score>maximum1:
            param = [upp,low,z]
            maximum1 = score

print('\nBest Model:')
print('parameters:',params)
print('R2:',maximum1)  

Fitting model 1 /63
Fitting model 2 /63
Fitting model 3 /63
Fitting model 4 /63
Fitting model 5 /63
Fitting model 6 /63
Fitting model 7 /63
Fitting model 8 /63
Fitting model 9 /63
Fitting model 10 /63
Fitting model 11 /63
Fitting model 12 /63
Fitting model 13 /63
Fitting model 14 /63
Fitting model 15 /63
Fitting model 16 /63
Fitting model 17 /63
Fitting model 18 /63
Fitting model 19 /63
Fitting model 20 /63
Fitting model 21 /63
Fitting model 22 /63
Fitting model 23 /63
Fitting model 24 /63
Fitting model 25 /63
Fitting model 26 /63
Fitting model 27 /63
Fitting model 28 /63
Fitting model 29 /63
Fitting model 30 /63
Fitting model 31 /63
Fitting model 32 /63
Fitting model 33 /63
Fitting model 34 /63
Fitting model 35 /63
Fitting model 36 /63
Fitting model 37 /63
Fitting model 38 /63
Fitting model 39 /63
Fitting model 40 /63
Fitting model 41 /63
Fitting model 42 /63
Fitting model 43 /63
Fitting model 44 /63
Fitting model 45 /63
Fitting model 46 /63
Fitting model 47 /63
Fitting model 48 /63
F

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/arthur.pentecoste/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-d8037c1d18f4>", line 12, in <module>
    scores=cross_val_score(estimator=model_0, X=X_alg_train, y=y_train, cv=5, scoring='r2')
  File "/Users/arthur.pentecoste/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 390, in cross_val_score
    error_score=error_score)
  File "/Users/arthur.pentecoste/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 236, in cross_validate
    for train, test in cv.split(X, y, groups))
  File "/Users/arthur.pentecoste/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 1007, in __call__
    while self.dispatch_one_batch(iterator):
  File "/Users/arthur.pentecoste/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py"

KeyboardInterrupt: 

## Final Model

In [ ]:
z = 10
upp, low = 0.9, 0.2
X_alg_train, X_alg_test = learn_structure(upp = 0.8, low = 0.1, 
                                          num_clusters = z, print_structure = False)

#scaler = MinMaxScaler()
#X_alg_train = scaler.fit_transform(X_alg_train)
#X_alg_test = scaler.transform(X_alg_test)
model_0 = LinearRegression().fit(X_alg_train, y_train)

print('Parameters:', upp, low, z) 
print('R2 score:', r2_score(y_test, model_0.predict(X_alg_test)))
print('MSE:', mean_squared_error(y_test, model_0.predict(X_alg_test)))
print('Number of coeff:', X_alg_train.shape[1])
print('MAPE:', mape(y_test, model_0.predict(X_alg_test)))

# Clustering

## K-Means

### Base model

In [0]:
z = 4
d = skuData[skuSet[0]]['X'].shape[1] #d is the number of columns
X_clus = np.zeros((skuSet.size, d))
count = 0
for sku in skuSet:
    X_clus[count, :] = np.mean(X_dict[sku]['train'], axis = 0)
    count += 1
    
kmeans = KMeans(n_clusters=z, random_state=0).fit(X_clus)

X_clus_train = np.zeros((train_size, d*z))
X_clus_test = np.zeros((test_size, d*z))

count = 0
for j in range(z):
    # the indices of items in cluster j
    clus_items = list(np.where(kmeans.labels_ == j)[0])
    for idx in clus_items:
        sku = skuSet[idx]
        # find the corresponding range
        idx_train = range_dict[sku]['train']
        idx_test = range_dict[sku]['test']

        # stack the data
        X_clus_train[idx_train[0]:idx_train[1], count:count+d] = X_dict[sku]['train']
        X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']


    count += d

model_clus = LinearRegression(fit_intercept=False).fit(X_clus_train, y_train)
print(r2_score(y_test, model_clus.predict(X_clus_test))) 
print('MSE:', mean_squared_error(y_test, model_clus.predict(X_clus_test)))

0.22768208216111396
MSE: 87379.35009750379


In [0]:
A=[[1,2],[3,4]]
np.mean(A,axis=0) #moyenne sur la colonne

array([2., 3.])

In [0]:
X_clus.shape #c'est du clustering sur les valeurs moyennes (pour toute une colonne, moyenne des valeurs des differents SKU)

(147, 64)

### Selection of number of clusters

In [0]:
num_clusters=0
maximum1=0

for z in range(2,10):
  d = skuData[skuSet[0]]['X'].shape[1] #d is the number of columns
  X_clus = np.zeros((skuSet.size, d))
  count = 0
  for sku in skuSet:
    X_clus[count, :] = np.mean(X_dict[sku]['train'], axis = 0)
    count += 1
    
  kmeans = KMeans(n_clusters=z, random_state=0).fit(X_clus)

  X_clus_train = np.zeros((train_size, d*z))
  X_clus_test = np.zeros((test_size, d*z))

  count = 0
  for j in range(z):
    # the indices of items in cluster j
    clus_items = list(np.where(kmeans.labels_ == j)[0])
    for idx in clus_items:
        sku = skuSet[idx]
        # find the corresponding range
        idx_train = range_dict[sku]['train']
        idx_test = range_dict[sku]['test']

        # stack the data
        X_clus_train[idx_train[0]:idx_train[1], count:count+d] = X_dict[sku]['train']
        X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

    count += d

  model_clus = LinearRegression(fit_intercept=False).fit(X_clus_train, y_train)
  score=r2_score(y_test, model_clus.predict(X_clus_test))

  if score>maximum1:
    num_clusters=z
    maximum1 = score

print('\nBest Model:')
print('number of clusters:',num_clusters)
print('R2:',maximum1)  


Best Model:
number of clusters: 9
R2: 0.29619304827390514


### Final Model

In [0]:
z = 9
d = skuData[skuSet[0]]['X'].shape[1] #d is the number of columns
X_clus = np.zeros((skuSet.size, d))
count = 0
for sku in skuSet:
    X_clus[count, :] = np.mean(X_dict[sku]['train'], axis = 0)
    count += 1
    
kmeans = KMeans(n_clusters=z, random_state=0).fit(X_clus)

X_clus_train = np.zeros((train_size, d*z))
X_clus_test = np.zeros((test_size, d*z))

count = 0
for j in range(z):
    # the indices of items in cluster j
    clus_items = list(np.where(kmeans.labels_ == j)[0])
    for idx in clus_items:
        sku = skuSet[idx]
        # find the corresponding range
        idx_train = range_dict[sku]['train']
        idx_test = range_dict[sku]['test']

        # stack the data
        X_clus_train[idx_train[0]:idx_train[1], count:count+d] = X_dict[sku]['train']
        X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']


    count += d

model_clus = LinearRegression(fit_intercept=False).fit(X_clus_train, y_train)
print(r2_score(y_test, model_clus.predict(X_clus_test))) 
print('MSE:', mean_squared_error(y_test, model_clus.predict(X_clus_test)))

0.29619304827390514
MSE: 79628.08140981212


In [0]:
X_clus_train.shape

(12685, 576)

## DBSCAN

### Base Model

In [0]:
d = skuData[skuSet[0]]['X'].shape[1] #d is the number of columns
X_clus = np.zeros((skuSet.size, d))
count = 0
for sku in skuSet:
    X_clus[count, :] = np.mean(X_dict[sku]['train'], axis = 0)
    count += 1
    
dbscan = DBSCAN(eps=15, min_samples = 4).fit(X_clus)
labels2=dbscan.labels_
z=max(labels2)+1 #this is the number of clusters
num_noise=len(list(np.where(labels2 == -1)[0])) #number of noisy sku

X_clus_train = np.zeros((train_size, d*(z+num_noise)))
X_clus_test = np.zeros((test_size, d*(z+num_noise)))

count=0

#sku in one on the clusters
for j in range(z):
    # the indices of items in cluster j
    clus_items = list(np.where(labels2 == j)[0])
    for idx in clus_items:
        sku = skuSet[idx]
        # find the corresponding range
        idx_train = range_dict[sku]['train']
        idx_test = range_dict[sku]['test']

        # stack the data
        X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
        X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

    count += d

#noise
clus_items=list(np.where(labels2 == -1)[0])
for idx in clus_items:
  sku = skuSet[idx]
  # find the corresponding range
  idx_train = range_dict[sku]['train']
  idx_test = range_dict[sku]['test']

  # stack the data
  X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
  X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

  count += d

model_clus = LinearRegression(fit_intercept=False).fit(X_clus_train, y_train)
print('R2:',r2_score(y_test, model_clus.predict(X_clus_test))) 
print('MSE:', mean_squared_error(y_test, model_clus.predict(X_clus_test)))

R2: 0.16459982557069552
MSE: 94516.41950407506


### Selection of eps and min_samples

In [0]:
eps_ = list(np.arange(1,20,0.5))
ms_ = list(range(2,15))
params=[]
maximum1=0

for i in range (15):

  print('model number:',i+1)

  eps = random.choice(eps_)
  ms = random.choice(ms_)

  print('  parameters:',[eps,ms])

  d = skuData[skuSet[0]]['X'].shape[1] #d is the number of columns
  X_clus = np.zeros((skuSet.size, d))
  count = 0
  for sku in skuSet:
    X_clus[count, :] = np.mean(X_dict[sku]['train'], axis = 0)
    count += 1
    
  dbscan = DBSCAN(eps=eps, min_samples = ms).fit(X_clus)
  labels2=dbscan.labels_
  z=max(labels2)+1 #this is the number of clusters
  num_noise=len(list(np.where(labels2 == -1)[0])) #number of noisy sku

  X_clus_train = np.zeros((train_size, d*(z+num_noise)))
  X_clus_test = np.zeros((test_size, d*(z+num_noise)))

  count=0

  #sku in one on the clusters
  for j in range(z):
    # the indices of items in cluster j
    clus_items = list(np.where(labels2 == j)[0])
    for idx in clus_items:
        sku = skuSet[idx]
        # find the corresponding range
        idx_train = range_dict[sku]['train']
        idx_test = range_dict[sku]['test']

        # stack the data
        X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
        X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

    count += d

  #noise
  clus_items=list(np.where(labels2 == -1)[0])
  for idx in clus_items:
    sku = skuSet[idx]
    # find the corresponding range
    idx_train = range_dict[sku]['train']
    idx_test = range_dict[sku]['test']

    # stack the data
    X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
    X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

    count += d

  model_clus = LinearRegression(fit_intercept=False).fit(X_clus_train, y_train)
  score=r2_score(y_test, model_clus.predict(X_clus_test))
  print('  R2:',score)

  if score>maximum1:
    params = [eps,ms]
    maximum1 = score

print('\nBest Model:')
print('parameters:',params)
print('R2:',maximum1) 

model number: 1
  parameters: [19.5, 7]
  R2: 0.16465738774287497
model number: 2
  parameters: [4.0, 8]
  R2: 0.20303415099505895
model number: 3
  parameters: [11.5, 12]
  R2: 0.2156294336085337
model number: 4
  parameters: [11.0, 10]
  R2: 0.2156294336085337
model number: 5
  parameters: [7.0, 13]
  R2: 0.1883811640307992
model number: 6
  parameters: [10.5, 11]
  R2: 0.2156294336085337
model number: 7
  parameters: [17.5, 9]
  R2: 0.16465738774287497
model number: 8
  parameters: [9.0, 3]
  R2: 0.2237741536908402
model number: 9
  parameters: [14.0, 12]
  R2: 0.22151761359539235
model number: 10
  parameters: [9.5, 7]
  R2: 0.218348305717326
model number: 11
  parameters: [1.0, 4]
  R2: 0.3548464172551853
model number: 12
  parameters: [7.5, 14]
  R2: 0.1883811640307992
model number: 13
  parameters: [14.0, 6]
  R2: 0.16465738774287497
model number: 14
  parameters: [17.5, 10]
  R2: 0.16465738774287497
model number: 15
  parameters: [11.5, 13]
  R2: 0.21559007652525808

Best Model

### Final Model

In [0]:
d = skuData[skuSet[0]]['X'].shape[1] #d is the number of columns
X_clus = np.zeros((skuSet.size, d))
count = 0
for sku in skuSet:
    X_clus[count, :] = np.mean(X_dict[sku]['train'], axis = 0)
    count += 1
    
dbscan = DBSCAN(eps=2.5, min_samples = 4).fit(X_clus)
labels2=dbscan.labels_
z=max(labels2)+1 #this is the number of clusters
num_noise=len(list(np.where(labels2 == -1)[0])) #number of noisy sku

X_clus_train = np.zeros((train_size, d*(z+num_noise)))
X_clus_test = np.zeros((test_size, d*(z+num_noise)))

count=0

#sku in one on the clusters
for j in range(z):
    # the indices of items in cluster j
    clus_items = list(np.where(labels2 == j)[0])
    for idx in clus_items:
        sku = skuSet[idx]
        # find the corresponding range
        idx_train = range_dict[sku]['train']
        idx_test = range_dict[sku]['test']

        # stack the data
        X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
        X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

    count += d

#noise
clus_items=list(np.where(labels2 == -1)[0])
for idx in clus_items:
  sku = skuSet[idx]
  # find the corresponding range
  idx_train = range_dict[sku]['train']
  idx_test = range_dict[sku]['test']

  # stack the data
  X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
  X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

  count += d

model_clus = LinearRegression(fit_intercept=False).fit(X_clus_train, y_train)
print('R2:',r2_score(y_test, model_clus.predict(X_clus_test))) 
print('MSE:', mean_squared_error(y_test, model_clus.predict(X_clus_test)))

R2: 0.329834990904886
MSE: 75821.86247430134


## OPTICS

### Base Model

In [0]:
d = skuData[skuSet[0]]['X'].shape[1] #d is the number of columns
X_clus = np.zeros((skuSet.size, d))
count = 0
for sku in skuSet:
    X_clus[count, :] = np.mean(X_dict[sku]['train'], axis = 0)
    count += 1
    
optics = OPTICS(min_samples=20).fit(X_clus)
labels2=optics.labels_
z=max(labels2)+1 #this is the number of clusters
num_noise=len(list(np.where(labels2 == -1)[0])) #number of noisy sku

X_clus_train = np.zeros((train_size, d*(z+num_noise)))
X_clus_test = np.zeros((test_size, d*(z+num_noise)))

count=0

#sku in one on the clusters
for j in range(z):
    # the indices of items in cluster j
    clus_items = list(np.where(labels2 == j)[0])
    for idx in clus_items:
        sku = skuSet[idx]
        # find the corresponding range
        idx_train = range_dict[sku]['train']
        idx_test = range_dict[sku]['test']

        # stack the data
        X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
        X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

    count += d

#noise
clus_items=list(np.where(labels2 == -1)[0])
for idx in clus_items:
  sku = skuSet[idx]
  # find the corresponding range
  idx_train = range_dict[sku]['train']
  idx_test = range_dict[sku]['test']

  # stack the data
  X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
  X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

  count += d

model_clus = LinearRegression(fit_intercept=False).fit(X_clus_train, y_train)
print('R2:',r2_score(y_test, model_clus.predict(X_clus_test))) 
print('MSE:', mean_squared_error(y_test, model_clus.predict(X_clus_test)))

R2: 0.16573211444498448
MSE: 94388.31336582199


### Selection of min_samples

In [0]:
ms_ = list(range(1,30))
opt=0
maximum1=0

for i in range (15):

  print('model number:',i+1)

  ms = random.choice(ms_)

  print('  parameter:',ms)

  d = skuData[skuSet[0]]['X'].shape[1] #d is the number of columns
  X_clus = np.zeros((skuSet.size, d))
  count = 0
  for sku in skuSet:
    X_clus[count, :] = np.mean(X_dict[sku]['train'], axis = 0)
    count += 1
    
  optics = OPTICS(min_samples=50).fit(X_clus)
  labels2=optics.labels_
  z=max(labels2)+1 #this is the number of clusters
  num_noise=len(list(np.where(labels2 == -1)[0])) #number of noisy sku

  X_clus_train = np.zeros((train_size, d*(z+num_noise)))
  X_clus_test = np.zeros((test_size, d*(z+num_noise)))

  count=0

  #sku in one on the clusters
  for j in range(z):
    # the indices of items in cluster j
    clus_items = list(np.where(labels2 == j)[0])
    for idx in clus_items:
        sku = skuSet[idx]
        # find the corresponding range
        idx_train = range_dict[sku]['train']
        idx_test = range_dict[sku]['test']

        # stack the data
        X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
        X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

    count += d

  #noise
  clus_items=list(np.where(labels2 == -1)[0])
  for idx in clus_items:
    sku = skuSet[idx]
    # find the corresponding range
    idx_train = range_dict[sku]['train']
    idx_test = range_dict[sku]['test']

    # stack the data
    X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
    X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

    count += d

  model_clus = LinearRegression(fit_intercept=False).fit(X_clus_train, y_train)
  score=r2_score(y_test, model_clus.predict(X_clus_test))
  print('  R2:',score)

  if score>maximum1:
    opt = ms
    maximum1 = score

print('\nBest Model:')
print('optimal parameter:',opt)
print('R2:',maximum1) 

model number: 1
  parameter: 4
  R2: 0.16262763426116278
model number: 2
  parameter: 1
  R2: 0.16262763426116278
model number: 3
  parameter: 24
  R2: 0.16262763426116278
model number: 4
  parameter: 27
  R2: 0.16262763426116278
model number: 5
  parameter: 19
  R2: 0.16262763426116278
model number: 6
  parameter: 2
  R2: 0.16262763426116278
model number: 7
  parameter: 14
  R2: 0.16262763426116278
model number: 8
  parameter: 8
  R2: 0.16262763426116278
model number: 9
  parameter: 4
  R2: 0.16262763426116278
model number: 10
  parameter: 27
  R2: 0.16262763426116278
model number: 11
  parameter: 9
  R2: 0.16262763426116278
model number: 12
  parameter: 24
  R2: 0.16262763426116278
model number: 13
  parameter: 29
  R2: 0.16262763426116278
model number: 14
  parameter: 11
  R2: 0.16262763426116278
model number: 15
  parameter: 11
  R2: 0.16262763426116278

Best Model:
optimal parameter: 4
R2: 0.16262763426116278


### Final Model

In [0]:
d = skuData[skuSet[0]]['X'].shape[1] #d is the number of columns
X_clus = np.zeros((skuSet.size, d))
count = 0
for sku in skuSet:
    X_clus[count, :] = np.mean(X_dict[sku]['train'], axis = 0)
    count += 1
    
optics = OPTICS(min_samples=4).fit(X_clus)
labels2=optics.labels_
z=max(labels2)+1 #this is the number of clusters
num_noise=len(list(np.where(labels2 == -1)[0])) #number of noisy sku

X_clus_train = np.zeros((train_size, d*(z+num_noise)))
X_clus_test = np.zeros((test_size, d*(z+num_noise)))

count=0

#sku in one on the clusters
for j in range(z):
    # the indices of items in cluster j
    clus_items = list(np.where(labels2 == j)[0])
    for idx in clus_items:
        sku = skuSet[idx]
        # find the corresponding range
        idx_train = range_dict[sku]['train']
        idx_test = range_dict[sku]['test']

        # stack the data
        X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
        X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

    count += d

#noise
clus_items=list(np.where(labels2 == -1)[0])
for idx in clus_items:
  sku = skuSet[idx]
  # find the corresponding range
  idx_train = range_dict[sku]['train']
  idx_test = range_dict[sku]['test']

  # stack the data
  X_clus_train[idx_train[0]:idx_train[1], count:(count+d)] = X_dict[sku]['train']
  X_clus_test[idx_test[0]:idx_test[1], count:count+d] = X_dict[sku]['test']

  count += d

model_clus = LinearRegression(fit_intercept=False).fit(X_clus_train, y_train)
print('R2:',r2_score(y_test, model_clus.predict(X_clus_test))) 
print('MSE:', mean_squared_error(y_test, model_clus.predict(X_clus_test)))

R2: 0.4144023787818263
MSE: 66253.98476299841


# Other Methods on Decentralized data

## Decentralized-Lasso

In [0]:
# model fitting - b1 Lasso
y_pred = []
for i in skuSet:
    model_i = LassoCV(alphas=[0.01,0.1,1,10,100], fit_intercept=False).fit(X_dict[i]['train'] , y_dict[i]['train'])
    y_pred += list(model_i.predict(X_dict[i]['test']))

print('R2:',r2_score(y_test, np.array(y_pred)))
#print(r2_score(y_test, np.array(y_pred), multioutput = 'variance_weighted'))
print('MSE:', mean_squared_error(y_test, np.array(y_pred)))
#print('MAPE:', mape(y_test,np.array(y_pred)))

R2: 0.43862418532597247
MSE: 63513.55149694479


## Elasticnet-Decentralized

In [0]:
#MODEL fitting - b1 elasticnet :

y_pred = []
for i in skuSet:
    model_i = ElasticNetCV(alphas=[0.01,0.1,1,10,100], fit_intercept=False).fit(X_dict[i]['train'] , y_dict[i]['train'])
    y_pred += list(model_i.predict(X_dict[i]['test']))

print('R2:',r2_score(y_test, np.array(y_pred)))
print('MSE:', mean_squared_error(y_test, np.array(y_pred)))

R2: 0.45782258658731356
MSE: 61341.461757241195


# Decision tree

## Decentralized

### Selection of parameters



In [0]:
max_features_ = list(range(2,64)) #there are 73-9 features
max_depth_ = list(range(2,10))
params=[]
maximum1=0

for i in range (20):

  print('model number:',i+1)

  mf = random.choice(max_features_)
  md = random.choice(max_depth_)

  print('  parameters:',[mf,md])

  y_pred = []
  for i in skuSet:
    model_i = DecisionTreeRegressor(max_features=mf,
                                    max_depth=md
                                        ).fit(X_dict[i]['train'] , y_dict[i]['train'])
    y_pred += list(model_i.predict(X_dict[i]['test']))

  score=r2_score(y_test, np.array(y_pred))
  print('  R2:',score)

  if score>maximum1:
    params = [mf,md]
    maximum1 = score

print('\nBest Model:')
print('parameters:',params)
print('R2:',maximum1) 

model number: 1
  parameters: [51, 2]
  R2: 0.3777224446827576
model number: 2
  parameters: [59, 7]
  R2: 0.33487270407058867
model number: 3
  parameters: [52, 7]
  R2: 0.3660148826223658
model number: 4
  parameters: [62, 7]
  R2: 0.3810041734524523
model number: 5
  parameters: [16, 5]
  R2: 0.3127194068783158
model number: 6
  parameters: [55, 2]
  R2: 0.38841283860673936
model number: 7
  parameters: [43, 6]
  R2: 0.35997100010079996
model number: 8
  parameters: [48, 2]
  R2: 0.38504536799684164
model number: 9
  parameters: [31, 8]
  R2: 0.45427237151821964
model number: 10
  parameters: [63, 6]
  R2: 0.28821138464977236
model number: 11
  parameters: [34, 9]
  R2: 0.11343783767580518
model number: 12
  parameters: [34, 6]
  R2: 0.3791264217600877
model number: 13
  parameters: [47, 5]
  R2: 0.3722485894569185
model number: 14
  parameters: [35, 9]
  R2: 0.3534608976093071
model number: 15
  parameters: [48, 6]
  R2: 0.3706261690676692
model number: 16
  parameters: [55, 4]
  R

### Final Model

In [0]:
y_pred = []
for i in skuSet:
    model_i = DecisionTreeRegressor(max_features=31,
                                    max_depth=8
                                    ).fit(X_dict[i]['train'] , y_dict[i]['train'])
    y_pred += list(model_i.predict(X_dict[i]['test']))

print('R2:',r2_score(y_test, np.array(y_pred)))
print('MSE:', mean_squared_error(y_test, np.array(y_pred)))

R2: 0.2918082562311244
MSE: 80124.17281227885


## Centralized

### Selection of parameters

In [0]:
max_features_ = list(range(2,64)) #there are 73 features minus the first 9
max_depth_ = list(range(2,10))
params=[]
maximum1=0

for i in range (20):

  print('model number:',i+1)

  mf = random.choice(max_features_)
  md = random.choice(max_depth_)

  print('  parameters:',[mf,md])

  DT_cen = DecisionTreeRegressor(max_features=mf,
                                 max_depth=md
                                 ).fit(X_cen_train, y_train)

  score=r2_score(y_test, RF_cen.predict(X_cen_test))
  print('  R2:',score)

  if score>maximum1:
    params = [mf,md]
    maximum1 = score

print('\nBest Model:')
print('parameters:',params)
print('R2:',maximum1)  

model number: 1
  parameters: [59, 6]
  R2: 0.3945759970893308
model number: 2
  parameters: [60, 9]
  R2: 0.3945759970893308
model number: 3
  parameters: [2, 3]
  R2: 0.3945759970893308
model number: 4
  parameters: [61, 3]
  R2: 0.3945759970893308
model number: 5
  parameters: [41, 9]
  R2: 0.3945759970893308
model number: 6
  parameters: [23, 7]
  R2: 0.3945759970893308
model number: 7
  parameters: [49, 7]
  R2: 0.3945759970893308
model number: 8
  parameters: [32, 8]
  R2: 0.3945759970893308
model number: 9
  parameters: [43, 7]
  R2: 0.3945759970893308
model number: 10
  parameters: [9, 7]
  R2: 0.3945759970893308
model number: 11
  parameters: [46, 8]
  R2: 0.3945759970893308
model number: 12
  parameters: [60, 9]
  R2: 0.3945759970893308
model number: 13
  parameters: [28, 6]
  R2: 0.3945759970893308
model number: 14
  parameters: [22, 3]
  R2: 0.3945759970893308
model number: 15
  parameters: [44, 6]
  R2: 0.3945759970893308
model number: 16
  parameters: [32, 8]
  R2: 0.3945

### Final Model

In [0]:
DT_cen = DecisionTreeRegressor(max_features=59,
                                 max_depth=6
                                 ).fit(X_cen_train, y_train)
print('R2:',r2_score(y_test, DT_cen.predict(X_cen_test)))  
print('MSE:', mean_squared_error(y_test, DT_cen.predict(X_cen_test)))

R2: 0.3489352383782781
MSE: 73660.87776531438


# Random forest

## Decentralized

### Selection of parameters

In [0]:
max_features_ = list(range(2,64)) #there are 73-9 features
n_estimators_ = list(range(50,200))
max_depth_ = list(range(2,10))
params=[]
maximum1=0

for i in range (20):

  print('model number:',i+1)

  mf = random.choice(max_features_)
  ne = random.choice(n_estimators_)
  md = random.choice(max_depth_)

  print('  parameters:',[mf,ne,md])

  y_pred = []
  for i in skuSet:
    model_i = RandomForestRegressor(max_features=mf,
                                    n_estimators=ne,
                                    max_depth=md
                                        ).fit(X_dict[i]['train'] , y_dict[i]['train'])
    y_pred += list(model_i.predict(X_dict[i]['test']))

  score=r2_score(y_test, np.array(y_pred))
  print('  R2:',score)

  if score>maximum1:
    params = [mf,ne,md]
    maximum1 = score

print('\nBest Model:')
print('parameters:',params)
print('R2:',maximum1) 

model number: 1
  parameters: [0.1, 11, 177, 5]
  R2: 0.5953945062994416
model number: 2
  parameters: [0.1, 26, 66, 9]
  R2: 0.6308878675985556
model number: 3
  parameters: [0.1, 62, 81, 9]
  R2: 0.5397911645162428
model number: 4
  parameters: [0.1, 49, 165, 8]
  R2: 0.5641095564631398
model number: 5
  parameters: [0.1, 33, 71, 2]
  R2: 0.5557752016130146
model number: 6
  parameters: [0.1, 27, 134, 8]
  R2: 0.5997695402260855
model number: 7
  parameters: [0.1, 34, 150, 7]
  R2: 0.6003370819734972
model number: 8
  parameters: [0.1, 34, 191, 5]
  R2: 0.5869912132137787
model number: 9
  parameters: [0.1, 3, 128, 9]
  R2: 0.6169321731974524
model number: 10
  parameters: [0.1, 18, 138, 7]
  R2: 0.6207848660165711
model number: 11
  parameters: [0.1, 49, 165, 9]
  R2: 0.5766638660067989
model number: 12
  parameters: [0.1, 32, 174, 5]
  R2: 0.602500499649165
model number: 13
  parameters: [0.1, 29, 133, 4]
  R2: 0.5808611231208164
model number: 14
  parameters: [0.1, 29, 89, 7]
  R2

### Final Model

In [0]:
y_pred = []
for i in skuSet:
  model_i = RandomForestRegressor(max_features=26,
                                  n_estimators=66,
                                  max_depth=9
                                  ).fit(X_dict[i]['train'] , y_dict[i]['train'])
  y_pred += list(model_i.predict(X_dict[i]['test']))

print('R2:',r2_score(y_test, np.array(y_pred)))
print('MSE:', mean_squared_error(y_test, np.array(y_pred)))

R2: 0.6209300583575346
MSE: 42887.630051238964


## Centralized

### Selection of paramaters

In [0]:
max_features_ = list(range(2,64)) #there are 73 features minus the first 9
n_estimators_ = list(range(50,200))
max_depth_ = list(range(2,10))
params=[]
maximum1=0

for i in range (20):

  print('model number:',i+1)

  mf = random.choice(max_features_)
  ne = random.choice(n_estimators_)
  md = random.choice(max_depth_)

  print('  parameters:',[mf,ne,md])

  RF_cen = RandomForestRegressor(max_features=mf,
                                 n_estimators=ne,
                                 max_depth=md
                                 ).fit(X_cen_train, y_train)

  score=r2_score(y_test, RF_cen.predict(X_cen_test))
  print('  R2:',score)

  if score>maximum1:
    params = [mf,ne,md]
    maximum1 = score

print('\nBest Model:')
print('parameters:',params)
print('R2:',maximum1) 

model number: 1
  parameters: [55, 58, 9]
  R2: 0.41985331148907856
model number: 2
  parameters: [56, 58, 8]
  R2: 0.3851882736564364
model number: 3
  parameters: [60, 141, 4]
  R2: 0.3082901344166139
model number: 4
  parameters: [22, 118, 9]
  R2: 0.40431711057046493
model number: 5
  parameters: [52, 126, 5]
  R2: 0.32136459885745916
model number: 6
  parameters: [58, 146, 7]
  R2: 0.34720158554121283
model number: 7
  parameters: [33, 110, 6]
  R2: 0.37893180597018583
model number: 8
  parameters: [42, 55, 9]
  R2: 0.37579899010813256
model number: 9
  parameters: [9, 106, 9]
  R2: 0.4182883350457668
model number: 10
  parameters: [42, 199, 7]
  R2: 0.368425547504622
model number: 11
  parameters: [23, 144, 7]
  R2: 0.38883248603032583
model number: 12
  parameters: [48, 199, 9]
  R2: 0.38481199418610823
model number: 13
  parameters: [49, 70, 3]
  R2: 0.28613549732779153
model number: 14
  parameters: [21, 56, 5]
  R2: 0.380186600818158
model number: 15
  parameters: [7, 174, 5]

### Final Model

In [0]:
RF_cen = RandomForestRegressor(max_features=55,
                                 n_estimators=58,
                                 max_depth=9
                                 ).fit(X_cen_train, y_train)
print('R2:',r2_score(y_test, RF_cen.predict(X_cen_test)))
print('MSE:', mean_squared_error(y_test, RF_cen.predict(X_cen_test)))

R2: 0.3945759970893308
MSE: 68497.12364021492


# Gradient Boosting

## Decentralized

### Selection of parameters

In [0]:
learning_rate_ = [0.01, 0.05, 0.1, 0.5]
max_features_ = list(range(2,64)) #there are 73-9 features
n_estimators_ = list(range(50,200))
max_depth_ = list(range(2,10))
params=[]
maximum1=0

for i in range (20):

  print('model number:',i+1)

  lr = random.choice(learning_rate_)
  mf = random.choice(max_features_)
  ne = random.choice(n_estimators_)
  md = random.choice(max_depth_)

  print('  parameters:',[lr,mf,ne,md])

  y_pred = []
  for i in skuSet:
    model_i = GradientBoostingRegressor(learning_rate=lr,
                                        max_features=mf,
                                        n_estimators=ne,
                                        max_depth=md
                                        ).fit(X_dict[i]['train'] , y_dict[i]['train'])
    y_pred += list(model_i.predict(X_dict[i]['test']))

  score=r2_score(y_test, np.array(y_pred))
  print('  R2:',score)

  if score>maximum1:
    params = [lr,mf,ne,md]
    maximum1 = score

print('\nBest Model:')
print('parameters:',params)
print('R2:',maximum1) 

model number: 1
  parameters: [0.1, 23, 73, 5]
  R2: 0.5625290827383915
model number: 2
  parameters: [0.5, 31, 197, 9]
  R2: 0.49062760263433425
model number: 3
  parameters: [0.1, 56, 159, 5]
  R2: 0.5136928710363216
model number: 4
  parameters: [0.5, 21, 133, 7]
  R2: 0.41103492644472184
model number: 5
  parameters: [0.05, 46, 137, 8]
  R2: 0.500396132658124
model number: 6
  parameters: [0.05, 17, 181, 7]
  R2: 0.5469154417686408
model number: 7
  parameters: [0.05, 33, 76, 2]
  R2: 0.6159345588515879
model number: 8
  parameters: [0.1, 52, 119, 4]
  R2: 0.49076319133434576
model number: 9
  parameters: [0.5, 13, 69, 6]
  R2: 0.44289619370462696
model number: 10
  parameters: [0.5, 21, 143, 2]
  R2: 0.3746990734559279
model number: 11
  parameters: [0.01, 7, 189, 5]
  R2: 0.5998029602037018
model number: 12
  parameters: [0.01, 19, 149, 8]
  R2: 0.6166717368791989
model number: 13
  parameters: [0.5, 27, 179, 3]
  R2: 0.3099930920368671
model number: 14
  parameters: [0.1, 43, 16

### Final Model

In [0]:
y_pred = []
for i in skuSet:
    model_i = GradientBoostingRegressor(learning_rate=0.01,
                                        max_features=19,
                                        n_estimators=149,
                                        max_depth=8
                                        ).fit(X_dict[i]['train'] , y_dict[i]['train'])
    y_pred += list(model_i.predict(X_dict[i]['test']))

print('R2:',r2_score(y_test, np.array(y_pred)))
print('MSE:', mean_squared_error(y_test, np.array(y_pred)))

R2: 0.6176965153325826
MSE: 43253.470181975856


## Centralized

### Selection of parameters

In [0]:
learning_rate_ = [0.01, 0.05, 0.1, 0.5]
max_features_ = list(range(2,64)) #there are 73 features minus the first 9
n_estimators_ = list(range(50,200))
max_depth_ = list(range(2,10))
params=[]
maximum1=0

for i in range (20):

  print('model number:',i+1)

  lr = random.choice(learning_rate_)
  mf = random.choice(max_features_)
  ne = random.choice(n_estimators_)
  md = random.choice(max_depth_)

  print('  parameters:',[lr,mf,ne,md])

  GB_cen = model_i = GradientBoostingRegressor(learning_rate=lr,
                                        max_features=mf,
                                        n_estimators=ne,
                                        max_depth=md
                                        ).fit(X_cen_train, y_train)

  score=r2_score(y_test, GB_cen.predict(X_cen_test))
  print('  R2:',score)

  if score>maximum1:
    params = [lr,mf,ne,md]
    maximum1 = score

print('\nBest Model:')
print('parameters:',params)
print('R2:',maximum1) 

model number: 1
  parameters: [0.05, 34, 154, 9]
  R2: 0.19223077190733073
model number: 2
  parameters: [0.01, 59, 158, 3]
  R2: 0.28635748330585276
model number: 3
  parameters: [0.5, 18, 186, 2]
  R2: 0.275164603047628
model number: 4
  parameters: [0.05, 36, 188, 8]
  R2: 0.2754754827786501
model number: 5
  parameters: [0.01, 36, 56, 4]
  R2: 0.2171682209439818
model number: 6
  parameters: [0.01, 17, 151, 3]
  R2: 0.2705764310583413
model number: 7
  parameters: [0.1, 49, 175, 2]
  R2: 0.3807290031934699
model number: 8
  parameters: [0.01, 42, 51, 5]
  R2: 0.23513690212187455
model number: 9
  parameters: [0.5, 32, 190, 2]
  R2: 0.2692181692493456
model number: 10
  parameters: [0.5, 2, 180, 9]
  R2: 0.15687143962184336
model number: 11
  parameters: [0.05, 9, 68, 7]
  R2: 0.3996982852358302
model number: 12
  parameters: [0.1, 23, 153, 9]
  R2: 0.22538097297103965
model number: 13
  parameters: [0.1, 61, 195, 7]
  R2: 0.2606583465387159
model number: 14
  parameters: [0.01, 11,

### Final Model

In [0]:
GB_cen = GradientBoostingRegressor(learning_rate=0.05,
                                        max_features=9,
                                        n_estimators=68,
                                        max_depth=7
                                        ).fit(X_cen_train, y_train)
print('R2:',r2_score(y_test, GB_cen.predict(X_cen_test)))  
print('MSE:', mean_squared_error(y_test, GB_cen.predict(X_cen_test)))

R2: 0.4383358458708545
MSE: 63546.17396188278


# Neural Net

## Basic Net

In [0]:
mlp = MLPClassifier(hidden_layer_sizes=(30,30),max_iter=100).fit(X_cen_train,y_train)
print('R2:',r2_score(y_test, mlp.predict(X_cen_test)))  
print('MSE:', mean_squared_error(y_test, mlp.predict(X_cen_test)))

R2: -0.9352150123815055
MSE: 218948.47468255213


## Hyperparameters Selection

In [0]:
learning_rates = [0.01, 0.05, 0.1, 0.5]
num_neuron = list(range(2,50))
batch_sizes = list(range(4,50))
params=[]
maximum1=0

for i in range (15):

  print('model number:',i+1)

  toss=random.choice([1,2])
  if toss==1:
    ls = (random.choice(num_neuron),)
  else:
    ls=(random.choice(num_neuron),random.choice(num_neuron))

  lr = random.choice(learning_rates)
  batch_size = random.choice(batch_sizes)

  print('  parameters:',[ls,lr,batch_size])

  mlp=MLPClassifier(hidden_layer_sizes=ls,
              activation='relu',
              batch_size=batch_size, 
              learning_rate_init=lr, 
              max_iter=100)
  
  mlp.fit(X_cen_train,y_train)

  score=r2_score(y_test, mlp.predict(X_cen_test))
  print('  R2:',score)

  if score>maximum1:
    params = [ls,lr,batch_size]
    maximum1 = score

print('\nBest Model:')
print('parameters:',params)
print('R2:',maximum1)  

model number: 1
  parameters: [(18, 6), 0.05, 7]
  R2: -0.1009095662017867
model number: 2
  parameters: [(37, 4), 0.05, 11]
  R2: -0.1009095662017867
model number: 3
  parameters: [(5, 37), 0.1, 30]
  R2: -0.1028072215862883
model number: 4
  parameters: [(9, 17), 0.5, 47]
  R2: -0.1028072215862883
model number: 5
  parameters: [(36,), 0.05, 42]
  R2: -0.10243410994723612
model number: 6
  parameters: [(42,), 0.1, 42]
  R2: -0.10472255432629729
model number: 7
  parameters: [(8, 30), 0.01, 4]
  R2: 0.2250145380004489
model number: 8
  parameters: [(28, 34), 0.5, 49]
  R2: -0.0990295881727925
model number: 9
  parameters: [(43,), 0.5, 37]
  R2: -0.10472255432629729
model number: 10
  parameters: [(3, 15), 0.01, 49]
  R2: 0.06475526183036107
model number: 11
  parameters: [(29, 20), 0.05, 40]
  R2: -0.054544083806472576
model number: 12
  parameters: [(12,), 0.01, 29]
  R2: -0.36328904073581647
model number: 13
  parameters: [(49, 48), 0.5, 40]
  R2: -0.08290526690967748
model number: 1

## Final Net

In [0]:
ls,lr,batch_size=params

mlp=MLPClassifier(hidden_layer_sizes=(8, 30),
              activation='relu',
              batch_size=4, 
              learning_rate_init=0.01, 
              max_iter=1000).fit(X_cen_train,y_train)

print('R2:',r2_score(y_test, mlp.predict(X_cen_test)))  
print('MSE:', mean_squared_error(y_test, mlp.predict(X_cen_test)))

R2: 0.21487454749899038
MSE: 88828.382975388
